# Vodafone Ziggo: cluster analysis using the Skynet case


## Import data

   Import data from a local csv file

In [1]:
# load libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.metrics import silhouette_score

# make sure all graphical output from mathplotlib is shown
%matplotlib inline

In [2]:
# Are we in the right place?
os.getcwd()

'D:\\Users\\welsendoorn\\OneDrive - VodafoneZiggo\\Training'

In [3]:
df = pd.read_csv('Skynet_input_clustering_sample.csv' ,sep = ';', decimal = '.')

In [3]:
# Data is a different directory: specify where
in_path = os.path.join('D:\\Users\\welsendoorn\\OneDrive - VodafoneZiggo\\Training', 'Skynet_input_clustering_full.csv')

df_2 = pd.read_csv(in_path ,sep = ';', decimal = '.')

In [4]:
# check, do we have data ;)
df_2.head()

,customers_id,urbanisation,age,gender,NPS,subscription,subscription_add_on,customer_lifetime,GB_usage_month,service_calls_year,perc_payment_on_time,marketing_optin,account_login_3_months,mechanic_year,survey_changes,survey_news,survey_usage,survey_support,survey_payment
0,146113,1,16.0,M,8,Complete,no add-ons,less then 12 months,53,0,1.0,Y,N,0,2.315,3.677,2.540,4.485,3.910
1,414632,1,36.0,F,10,Internet only Compact,no add-ons,> 5 years,92,0,1.0,N,Y,0,4.843,2.583,4.783,1.800,2.835
2,377220,2,17.0,M,8,Complete,no add-ons,less then 12 months,40,0,1.0,N,N,0,1.775,4.859,1.991,4.599,3.946
3,244103,1,39.0,M,10,Internet only Compact,no add-ons,> 5 years,93,0,1.0,N,Y,1,2.676,3.355,4.299,3.473,3.072
4,450183,2,20.0,M,9,Internet & Television Plus,no add-ons,less then 12 months,43,0,1.0,Y,Y,0,3.363,4.200,4.104,4.523,3.280


In [11]:
# do we have all data?
df_2.shape

(27637, 19)

In [12]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27637 entries, 0 to 27636
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   customers_id            27637 non-null  int64  
 1   urbanisation            27637 non-null  int64  
 2   age                     26787 non-null  float64
 3   gender                  27637 non-null  object 
 4   NPS                     27637 non-null  int64  
 5   subscription            27637 non-null  object 
 6   subscription_add_on     27637 non-null  object 
 7   customer_lifetime       27637 non-null  object 
 8   GB_usage_month          27637 non-null  int64  
 9   service_calls_year      27637 non-null  int64  
 10  perc_payment_on_time    27637 non-null  float64
 11  marketing_optin         27637 non-null  object 
 12  account_login_3_months  27637 non-null  object 
 13  mechanic_year           27637 non-null  int64  
 14  survey_changes          26809 non-null

## Data preparation

- missing values
- outliers
- compute dummies for categories (can also be done in source)
- selection (correlation plot: do not select variables that correlate too high)
- normalization

### Missing values
K-means cannot handle missing values, so find out if your dataset has missing values and solve any NAs

In [13]:
# Do we have missing values?
df.isna().sum().sort_values()

customers_id                0
mechanic_year               0
account_login_3_months      0
marketing_optin             0
perc_payment_on_time        0
GB_usage_month              0
customer_lifetime           0
service_calls_year          0
subscription                0
NPS                         0
gender                      0
urbanisation                0
subscription_add_on         0
survey_news               277
survey_payment            294
survey_changes            394
age                       426
survey_support            436
survey_usage              459
dtype: int64

In [ ]:
# If you do encounter missing values you can either remove the record or substitute it 
# with another value, like the mean or median. inplace=True > replace original, else a copy will be made.
df['survey_news'].fillna((df['survey_news'].mean()), inplace=True)

# Replace missing values on age with the mean
df['age'].fillna((df['age'].mean()), inplace=True)

# Make sure age is an integer and not a float
df['age'] = df['age'].astype('int64')

df = df.dropna()

### Inspect your data and deal with outliers

In [ ]:
# Use a boxplot to inspect your data
sns.boxplot(x=df['age'])

In [ ]:
# Or use the histogram function in pandas
df['age'].hist(bins=100)

In [ ]:
# Alternative: use a scatterplot against another variable
sns.scatterplot(x=df['age'], y=df['survey_news'])

In [ ]:
# You can use a function to remove all values above or below 1.5 times the IQR, or replace values you see fit
# Here we substitute people with age 0 with the mean
df['age'] = np.where(df['age'] == 0 , df['age'].mean(), df['age']).astype('int64')

# And what about people older than 80 years?
df['age'] = np.where(df['age'] >=80 , 80, df['age']).astype('int64')

# Check results
sns.boxplot(x=df['age'])

### Create numeric variables as input
K-mean can handle only numeric input, so we need to dummify categories (if you have them in your data).

In [ ]:
# Which variabels are not numeric?
df.describe(exclude=[np.number])

In [ ]:
# Create a new dataframe with all the input for the cluster analysis, starting with all numeric columns
df_input = df.select_dtypes(include = 'number')

# Show
df_input.head()

In [ ]:
# Specify which variables you want to dummify, example
pd.get_dummies(df['gender'], prefix='gender', drop_first=False, dummy_na= False)

In [ ]:
# Add dummified data for gender to your dataframe
df_input = pd.concat([df_input, pd.get_dummies(df['gender'], prefix='gender', drop_first=False, dummy_na= False)],axis=1)

# Add dummified data for online login to your dataframe
df_input = pd.concat([df_input, pd.get_dummies(df['marketing_optin'], prefix='optin', drop_first=False, dummy_na= False)],axis=1)

# Add dummified data for online login to your dataframe
df_input = pd.concat([df_input, pd.get_dummies(df['account_login_3_months'], prefix='login', drop_first=False, dummy_na= False)],axis=1)

df_input.head()

## K-means clustering

- choose variables that are input for clustering > not to high corrrelations
- determine # clusters using elbow measure or silhouette score
- difference between normalized data and raw data

In [ ]:
# Can we add all data, first check for high correlations
corrMatrix = df_input.select_dtypes(include = 'number').corr()

# Clearly number of koffie and number of orders correlate too high
plt.figure(figsize=(16,10))
corrMatrixDisplay = sns.heatmap(corrMatrix, annot=True, linewidths=2, cmap='coolwarm', fmt='.2f')
corrMatrixDisplay.set_ylim(sorted(corrMatrixDisplay.get_xlim(), reverse=True)) # fit top & bottom limits for mathplotlib

In [ ]:
# Drop the variables we do not use because of to high correlation
df_input.drop(columns = ['gender_M', 'login_N', 'optin_N'], inplace = True)

In [ ]:
# Standardize the input, clusters will tend to be separated along variables with greater variance

# define standard scaler
scaler = StandardScaler()

# save the ID field, we don't want to scale it
ID = df_input['customers_id']

# fit scaler on all remaining fields
scaler.fit(df_input.drop('customers_id', axis = 1))

# Calculate scaled values and store them in a separate object
scaled_values = scaler.transform(df_input.drop('customers_id', axis = 1))

# Create a dataframe, add column names and put ID field back
df_scaled = pd.DataFrame(scaled_values, index = df_input.index, columns = df_input.drop('customers_id', axis = 1).columns)
df_scaled['customers_id'] = ID
df_scaled.head()

As you know for cluster analysis, an unsupervised method, there is no 'best' outcome. You have to evaluate cluster solutions based on a few metrics but foremost your business knowledge. **Guidelines**:

- Limited number of clusters 
- That differ significantly
- Are large enough to handle for our case
- Are stable: repeated attempts (even on samples) lead to the same result
- Cluster membership is easy to profile: expressive features that enable labelling

In [ ]:
# Select initial set of variables for kmeans input
kmeans_input = df_scaled[['GB_usage_month', 'perc_payment_on_time', 'survey_support', 'survey_changes', 'survey_usage', 'age', 'login_Y']]


In [ ]:
# Use the elbow visual as guide to decide how many clusters we want to use; WCSS within cluster sum of squares
wcss = []

for i in range(1, 12):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=4343, algorithm='auto')
    kmeans.fit(kmeans_input)
    wcss.append(kmeans.inertia_)
plt.plot(range(1, 12), wcss)
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

In [ ]:
# Use the silhouette measure as guide to decide how many clusters we want to use
silhouette_coefficients = []

for k in range(2, 12):
    kmeans = KMeans(n_clusters=k, init='k-means++', max_iter=300, n_init=10, random_state=4343, algorithm='auto'))
    kmeans.fit(kmeans_input)
    score = silhouette_score(kmeans_input, kmeans.labels_, metric='euclidean')
    silhouette_coefficients.append(score)
plt.plot(range(2, 12), silhouette_coefficients)
plt.xticks(range(2, 12))
plt.xlabel("Number of Clusters")
plt.ylabel("Silhouette Coefficient")
plt.show()

In [ ]:
# Set k-means to 3 clusters
kmeans = KMeans(n_clusters=3, init='k-means++', max_iter=300, n_init=10, random_state=4343, algorithm='auto')) 
kmeans.fit(kmeans_input)

# Save clusters
clusters = kmeans.fit_predict(kmeans_input)

In [ ]:
# Save cluster solution in your dataframe, you can alsways enter multiple solutions
df_input['cluster_solution'] = pd.Series(clusters, index=df_input.index)


## Evaluate 

- size of clusters
- visualize results of clusters > dimension reduction
- plot cluster solution against various different characteristics
- interprete meaning: is this solution usefull for our task?


In [ ]:
# How big are our clusters?
df_input['cluster_solution'].value_counts()

In [ ]:
# Visualise cluster solution along two dimensions (PCA)
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca_result = pca.fit_transform(kmeans_input.values)

In [ ]:
# Add dimensions to our input dataframe
df_input['pca-1'] = pca_result[:,0]
df_input['pca-2'] = pca_result[:,1]
plt.figure(figsize=(16,10))
sns.scatterplot(
    x="pca-1", y="pca-2",
    hue="cluster_solution",
    palette=sns.color_palette("hls", 3),
    data=df_input,
    legend="full",
    alpha=0.4
)

In [ ]:
# Look at mean differences between clusters
df_input.groupby(
   ['cluster_solution']
).agg(
    {
         'customers_id': 'count',
         'age': 'mean',                
         'urbanisation' : 'mean',
         'GB_usage_month': 'mean',
         'service_calls_year': 'mean',
         'perc_payment_on_time': 'mean',
         'mechanic_year': 'mean',
         'NPS': 'mean',
         'optin_Y': 'mean',
         'login_Y': 'mean',
         'gender_F': 'mean',
         'survey_news': 'mean',
         'survey_usage': 'mean', 
         'survey_support': 'mean',
         'survey_payment': 'mean', 
         'survey_changes': 'mean' 
    }
).reset_index()

In [ ]:
# Looking at the mean value is sometimes not that informative, display the distribution of age for example
sns.FacetGrid(df_input
              ,hue="cluster_solution"
              ,height=5, aspect=2).map(sns.distplot, "age", label="cluster_solution", hist=False, kde_kws={"shade": True})
plt.legend(title="Cluster number")
plt.title('Distribution of customer age')


In [ ]:
# Display the distribution of GB usage per month
sns.FacetGrid(df_input
              ,hue="cluster_solution"
              ,height=5, aspect=2).map(sns.distplot, "survey_support", label="cluster_solution", hist=False, kde_kws={"shade": True})
plt.legend(title="Cluster number")
plt.title('Distribution of website usage for online support')

In [ ]:
# Display the distribution of GB usage per month
sns.FacetGrid(df_input
              ,hue="cluster_solution"
              ,height=5, aspect=2).map(sns.distplot, "GB_usage_month", label="cluster_solution", hist=False, kde_kws={"shade": True})
plt.legend(title="Cluster number")
plt.title('Distribution of GB usage per month')


## Save results

- Merge other features for exploratory analysis
- Save file to CSV


In [ ]:
# Create ouput dataframe: non-numeric fields to keep and Kmeans input
df_save = pd.merge(df[['customers_id', 'subscription', 'subscription_add_on', 'customer_lifetime']], df_input, on='customers_id')

# Subset final dataframe and upload to SQL-server
df_save.to_csv('wvg_cluster_solution.csv', index = False, sep = ';', decimal = ',')